**Specifying the Schema Name**:

Most databases support the concept of multiple “schemas” -namespaces that refer to alternate sets of tables and other constructs. The server-side geometry of a “schema” takes many forms, including names of “schemas” under the scope of a particular database (e.g. PostgreSQL schemas), named sibling databases (e.g. MySQL / MariaDB access to other databases on the same server) and ....

What all of the above approaches have (mostly) in *common* is that there’s a way of *referring to this alternate set of tables using a string name.* SQLAlchemy refers to this name as the schema name.

Within SQLAlchemy, this is *nothing more than a string name* which is associated with a Table object,and is then rendered into SQL statements in a manner appropriate to the target database such that the table is referred towards in its remote “schema”, whatever mechanism that is on the target database.

The “schema” name may be *associated directly with a Table* using the Table.schema argument; when using the ORM with declarative table configuration, the parameter is passed using the __table_args__ parameter dictionary.

The “schema” name may also be *associated with the MetaData* object where it will take effect automatically for all Table objects associated with that MetaData that don’t otherwise specify their own name. 

**What’s “schema” ?**

SQLAlchemy’s support for database “schema” was designed with first party support for PostgreSQL-style schemas. In this style, there is first a “database” that typically has a single “owner”. Within this database there can be any number of “schemas” which then contain the actual table objects.

A table within a specific schema is referred towards explicitly using the syntax “{schemaname}.{tablename}”.Contrast this to an architecture such as that of MySQL, where there are only “databases”, however SQL statements can refer to multiple databases at once, using the same syntax except it is “{database}.{tablename}”. On Oracle, this syntax refers to yet another concept, the “owner” of a table. Regardless of which kind of database is in use, SQLAlchemy uses the phrase “schema” to refer to the qualifying identifier within the general syntax of “{qualifier}.{tablename}”. 

The most basic example is that of the Table.schema argument using a Core Table object as follows:

In [ ]:
metadata_obj = MetaData()

financial_info = Table(
    "financial_info",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("value", String(100), nullable=False),
    schema="remote_banks",
)

print(select(financial_info))

SQL that is rendered using this Table, such as the SELECT statement below, will explicitly qualify the table name *financial_info* with the *remote_banks* schema name:

In [ ]:
SELECT remote_banks.financial_info.id, remote_banks.financial_info.value
FROM remote_banks.financial_info

When a Table object is declared with an explicit schema name, it is stored in the **internal MetaData namespace** using the combination of the schema and table name. 

This dotted name is also what *must be used* when referring to the table for use with the **ForeignKey or ForeignKeyConstraint** objects, even if the referring table is also in that same schema:

In [ ]:
customer = Table(
    "customer",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("financial_info_id", ForeignKey("remote_banks.financial_info.id")),
    schema="remote_banks",
)

The Table.schema argument may also be used with certain dialects to indicate a multiple-token (e.g. dotted) path to a particular table. This is particularly important on a database such as Microsoft SQL Server where there are often dotted “database/owner” tokens. The tokens may be placed directly in the name at once, such as:

schema = "dbo.scott"